step 2 : translate.ipynb (NLLB 번역 파트)

In [ ]:
from transformers import pipeline

# 1. 번역 파이프라인 준비
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

Device set to use cpu


In [ ]:
# 2. 원본 텍스트 로드
with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
    original_text = f.read()

In [7]:
result = translator('''
aapl-20230701

1

UNITED STATES

1

SECURITIES AND EXCHANGE COMMISSION

1

Washington, D.C. 20549

1

FORM 10-Q

1

(Mark One)

1

QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

1

or

1

TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

1

For the transition period from

1

to

1

Commission File Number:

1

Apple Inc.

1

(Exact name of Registrant as specified in its charter)

1

California

''', src_lang="en", tgt_lang="kor_Hang", 
num_beams=1,             
penalty_alpha=None)[0]['translation_text']         
result

'aapl-20230701 1 미국 1 증권 및 거래소 위원회 1 워싱턴 DC 20549 1 양식 10-Q 1 (Marken One) 1 방정식 보고서 SECTION 13 또는 15에 관한 법률 1 또는 1 전환 보고서 1 또는 1934 SECURITIES EXCHANGE ACT의 SECTION 13 또는 15에 관한 법률 1 또는 1 전환 보고서 1 미국 증권 거래소 법 1934의 SECURITIES EXCHANGE ACT의 전환 기간 1 1에서 1 파일 번호: 1 애플 Inc. 1 (기본에 명시된 등록자의 정확한 이름) 1 캘리포니아 '

In [10]:
import os
from tqdm import tqdm
from transformers import pipeline

# 파일 경로
input_path = "data/segments/AAPL_2023_Q3_joined.txt"
output_path = "data/translations/AAPL_2023_Q3_joined_en.txt"

# 번역기 로딩
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

# 1. 텍스트 불러오기 및 분할
with open(input_path, "r", encoding="utf-8") as f:
    content = f.read()

segments = content.strip().split("\n\n1\n\n")

# 2. 의미 단위로 묶기 (예: 10개씩 묶기)
chunk_size = 10
chunks = [
    "\n\n1\n\n".join(segments[i:i+chunk_size])
    for i in range(0, len(segments), chunk_size)
]

# 3. tqdm을 통해 진행 상황 시각화하며 번역
translated_chunks = []
for chunk in tqdm(chunks, desc="🔁 Translating chunks"):
    result = translator(
        chunk.strip(),
        src_lang="en",
        tgt_lang="kor_Hang",
        max_length=1024,
        num_beams=4
    )[0]["translation_text"]
    translated_chunks.append(result)

# 4. 다시 개별 segment 단위로 분할
translated_segments = []
for chunk in translated_chunks:
    translated_parts = chunk.split("\n\n1\n\n")
    translated_segments.extend(translated_parts)

# ⚠️ 디버깅용 경고 메시지
if len(translated_segments) != len(segments):
    print(f"⚠️ 경고: 번역 후 segment 수가 다릅니다. 원본 {len(segments)}개, 번역본 {len(translated_segments)}개")

# 5. 결과 저장
final_output = "\n\n1\n\n".join(translated_segments)
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    f.write(final_output)

print(f"✅ 저장 완료: {output_path}")


Device set to use cpu
🔁 Translating chunks:  14%|█▍        | 10/70 [02:26<14:40, 14.67s/it]


KeyboardInterrupt: 

In [ ]:
import os
from tqdm import tqdm
from transformers import pipeline

# 파일 경로
input_path = "data/segments/AAPL_2023_Q3_joined.txt"
output_path = "data/translations/AAPL_2023_Q3_joined_en.txt"

# 번역기 로딩
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

# 1. 텍스트 불러오기 및 분할
with open(input_path, "r", encoding="utf-8") as f:
    content = f.read()

segments = content.strip().split("\n\n1\n\n")

# 2. 의미 단위로 묶기 (예: 10개씩 묶기)
chunk_size = 10
chunks = [
    "\n\n1\n\n".join(segments[i:i+chunk_size])
    for i in range(0, len(segments), chunk_size)
]

# 3. tqdm + 원본/번역 결과 콘솔 출력
translated_chunks = []
for idx, chunk in enumerate(tqdm(chunks, desc="🔁 Translating chunks", unit="chunk")):
    # 번역 실행
    result = translator(
        chunk.strip(),
        src_lang="en",
        tgt_lang="kor_Hang",
        max_length=1024,
        num_beams=4
    )[0]["translation_text"]

    translated_chunks.append(result)

    # 콘솔 출력
    print(f"\n🧩 Chunk {idx+1}/{len(chunks)}")
    print("📄 [원본]:\n", chunk.strip(), "원본끝!!\n\n")  # 길면 일부만 출력 (300자 제한)
    print("🌐 [번역]:\n", result.strip(), "번역끝!!\n\n")  # 마찬가지로 일부만 출력
    print("-" * 80)

# 4. 다시 개별 segment 단위로 분할
translated_segments = []
for chunk in translated_chunks:
    translated_parts = chunk.split("\n\n1\n\n")
    translated_segments.extend(translated_parts)

# ⚠️ 경고: segment 개수 불일치 확인
if len(translated_segments) != len(segments):
    print(f"⚠️ 경고: 번역 후 segment 수가 다릅니다. 원본 {len(segments)}개, 번역본 {len(translated_segments)}개")

# 5. 저장
final_output = "\n\n1\n\n".join(translated_segments)
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    f.write(final_output)

print(f"✅ 저장 완료: {output_path}")


Device set to use cpu
🔁 Translating chunks:   1%|▏         | 1/70 [00:12<14:32, 12.64s/chunk]


🧩 Chunk 1/70
📄 [원본]:
 aapl-20230701

1

UNITED STATES

1

SECURITIES AND EXCHANGE COMMISSION

1

Washington, D.C. 20549

1

FORM 10-Q

1

(Mark One)

1

QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

1

or

1

TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

1

For the transition period from 원본끝!!


🌐 [번역]:
 aapl-20230701 1 미국 1 증권 및 거래소 위원회 1 워싱턴 D.C. 20549 1 양식 10-Q 1 (Mark One) 1 분기 보고서는 SECTION 13 또는 15에 관한 법률의 1 또는 1 전환 보고서는 SECTION 13 또는 15에 관한 법률의 1 또는 1 전환 보고서는 SECTION 13 또는 15에 관한 법률의 1 전환 기간에 관한 법률의 1 번역끝!!


--------------------------------------------------------------------------------


🔁 Translating chunks:   3%|▎         | 2/70 [00:21<11:58, 10.56s/chunk]


🧩 Chunk 2/70
📄 [원본]:
 to

1

Commission File Number:

1

Apple Inc.

1

(Exact name of Registrant as specified in its charter)

1

California

1

(State or other jurisdiction

1

of incorporation or organization)

1

(I.R.S. Employer Identification No.)

1

One Apple Park Way

1

Cupertino, California 원본끝!!


🌐 [번역]:
 1 to 1 위원회 파일 번호: 1 Apple Inc. 1 (이 법령에서 명시된 등록자의 정확한 이름) 1 캘리포니아 1 (주 또는 다른 관할 구역 1 법인 설립 또는 조직) 1 (I.R.S. 고용주 식별 번호) 1 One Apple Park Way 1 쿠퍼티노, 캘리포니아 번역끝!!


--------------------------------------------------------------------------------


🔁 Translating chunks:   4%|▍         | 3/70 [00:32<11:46, 10.55s/chunk]

...


🧩 Chunk 65/70
📄 [원본]:
 Other Legal Proceedings

1

Apple Inc. | Q3 2023 Form 10-Q | 20

1

Purchases of Equity Securities by the Issuer and Affiliated Purchasers

1

Periods

1

Total Number

1

of Shares Purchased

1

Average Price

1

Paid Per Share

1

Total Number of Shares

1

Purchased as Part of Publicly 원본끝!!


🌐 [번역]:
 다른 법적 절차 1 Apple Inc. Q3 2023 양식 10-Q 20 1 발행자 및 계열 구매자의 주식 증권 구매 1 기간 1 총 1 주식 구매 1 평균 가격 1 주식 지불 1 주식 구매 1 주식 구매 1 주식 구매 번역끝!!


--------------------------------------------------------------------------------


🔁 Translating chunks:  94%|█████████▍| 66/70 [25:12<01:29, 22.28s/chunk]


🧩 Chunk 66/70
📄 [원본]:
 Announced Plans or Programs

1

Approximate Dollar Value of

1

Shares That May Yet Be Purchased

1

Under the Plans or Programs

1

April 2, 2023 to May 6, 2023:

1

Open market and privately negotiated purchases

1

May 7, 2023 to June 3, 2023:

1

Open market and privately negotiated purchases

1

June 4, 2023 to July 1, 2023:

1

Open market and privately negotiated purchases 원본끝!!


🌐 [번역]:
 발표된 계획 또는 프로그램 1 아직 구매될 수 있는 1 주식의 약 달러 가치 1 계획 또는 프로그램 하에서 2023년 4월 2일부터 2023년 5월 6일까지: 1 오픈 시장 및 민간 협상 구매 1 5월 7일부터 2023년 6월 3일까지: 1 오픈 시장 및 민간 협상 구매 1 6월 4일부터 2023년 7월 1일까지: 1 오픈 시장 및 민간 협상 구매 번역끝!!


--------------------------------------------------------------------------------


🔁 Translating chunks:  96%|█████████▌| 67/70 [25:18<00:52, 17.45s/chunk]


🧩 Chunk 67/70
📄 [원본]:
 Total

1

None.

1

Not applicable.

1

None

1

Apple Inc. | Q3 2023 Form 10-Q | 21

1

Incorporated by Reference

1

Exhibit

1

Number

1

Exhibit Description

1

Form 원본끝!!


🌐 [번역]:
 전체 1 None. 1 적용되지 않습니다. 1 None 1 Apple Inc.en Q3 2023 양식 10-Q 번역끝!!


--------------------------------------------------------------------------------


In [44]:

# # HTML 파일 불러오기
# # load original segments
# with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
#     segments = f.read().split("\n\n␟\n\n")

# translations = []
# for seg in segments:
#     if seg.strip():
#         result = translator(seg, src_lang="en", tgt_lang="kor_Hang",num_beams=2, penalty_alpha=0.6, top_k=4,)[0]['translation_text']
#     else:
#         result = ""
#     translations.append(result)

# # save to file
# with open("data/translations/AAPL_2023_Q3_translated.txt", "w", encoding="utf-8") as f:
#     f.write("\n\n␟\n\n".join(translations))


In [5]:
# key: value  
# <eng>: korean
translation_cache_table = {}

In [15]:
result = translator("APPL", src_lang="en", tgt_lang="kor_Hang",num_beams=2, penalty_alpha=0.6, top_k=1, top_p=1, do_sample=True)[0]['translation_text']
result

'APPL에서 APPL을 사용합니다.'

In [42]:
result[:100]

'aapl-20230701  미국 증권 및 거래소위원회  워싱턴 DC 20549  FORM 10-Q  (Mark One)  SECURITIES EXCHANGE ACT OF 1934 '

두번째태그달아서해보기시도


In [77]:
# 1. load and split by ␟
with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
    original = f.read()

segments = original.strip().split("\n\n␟\n\n")  # 구조 보존용
tagged_segments = [f"<SEG{i}>{seg}" for i, seg in enumerate(segments)]
tagged_text = "\n".join(tagged_segments)


In [78]:
from transformers import pipeline

translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

translated_text = translator(
    tagged_text,
    src_lang="en",
    tgt_lang="kor_Hang",
    do_sample=False,
    num_beams=4,
    max_length=1024  # 필수!
)[0]["translation_text"]


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (10506 > 1024). Running this sequence through the model will result in indexing errors
Your input_length: 10506 is bigger than 0.9 * max_length: 1024. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [79]:
import re

# 태그별로 추출 (예: <SEG0> ... <SEG1> ...)
matches = re.split(r"<SEG\d+>", translated_text)
# 앞부분이 공백이거나 ""일 수 있으므로 제거
translated_segments = [seg.strip() for seg in matches if seg.strip()]

# 원래 순서 유지
for i, (orig, trans) in enumerate(zip(segments, translated_segments)):
    print(f"[{i}] {orig[:30]} → {trans[:30]}")


[0] aapl-20230701 → <SEGSEGSEGSEGSEGSEGSEGSEGSEGSE


In [ ]:
with open("data/translations/AAPL_2023_Q3_translated_list.txt", "w", encoding="utf-8") as f:
    for seg in translated_segments:
        f.write(seg + "\n\n␟\n\n")


몰라 ==-----=====


In [ ]:
from tqdm import tqdm

In [39]:
translations = []
for seg in tqdm(segments[:10]):
    if seg.strip():
        trans_result = translator(seg, src_lang="en", tgt_lang="kor_Hang", 
                    num_beams=5,             
                    penalty_alpha=None)     
        result = trans_result[0]['translation_text']
        
        
    else:
        result = ""
    translations.append(result)
    
dict(zip(segments[:10], translations))

 10%|█         | 1/10 [00:01<00:16,  1.82s/it]

[{'translation_text': 'aapl-20230701에 대한'}]


 20%|██        | 2/10 [00:04<00:17,  2.22s/it]

[{'translation_text': '미국에서는 물론이고, 미국에서는 물론이고, 미국에서도 마찬가지입니다.'}]


 30%|███       | 3/10 [00:07<00:20,  2.87s/it]

...

[{'translation_text': '(Mark One에서)'}]


 70%|███████   | 7/10 [00:16<00:07,  2.56s/it]

[{'translation_text': 'SECURITIES EXCHANGE ACT OF 1934 제13조 또는 15조의 4차 보고서를 제출합니다.'}]


 70%|███████   | 7/10 [00:22<00:09,  3.26s/it]


KeyboardInterrupt: 

{'aapl-20230701': 'aapl-20230701에 대한 설명',
 'UNITED STATES': '미국에서는 미군이',
 'SECURITIES AND EXCHANGE COMMISSION': '증권 및 교환위원회',
 'Washington, D.C. 20549': '워싱턴 D.C. 20549에 대한',
 'FORM 10-Q': 'FORM 10-Q의 형태는',
 '(Mark One)': '(Mark One) (Mark One)',
 'QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934': '13조 또는 15조에 따른 분기보고 (주) (d) 1934년 미국 증권거래법',
 'or': '또는 오렌지',
 'TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934': '13조 또는 15조에 의한 전환 보고서를 제출 (D) 1934년 미국 보안 거래소 법의',
 'For the transition period from': '이 기간 동안은 전환 기간이'}

In [38]:
translator("UNITED STATES", src_lang="en", tgt_lang="kor_Hang", 
                   do_sample=False,        # 샘플링 비활성화 → 핵심

                    num_beams=1,   
                    top_k = 1,
                    top_p = 1 ,      
                    penalty_alpha=None)[0]['translation_text']  

The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'미국에서는 미군이'

In [29]:
translations

['aapl-20230701에 대한 설명',
 '미국에서는 미군이',
 '증권 및 교환위원회',
 '워싱턴 D.C. 20549에 대한',
 'FORM 10-Q의 형태는',
 '(Mark One) (Mark One)',
 '13조 또는 15조에 따른 분기보고 (주) (d) 1934년 미국 증권거래법',
 '또는 오렌지',
 '13조 또는 15조에 의한 전환 보고서를 제출 (D) 1934년 미국 보안 거래소 법의',
 '이 기간 동안은 전환 기간이']

In [ ]:

# HTML 파일 불러오기
# load original segments
with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
    segments = f.read().split("\n\n␟\n\n")

translations = []
for seg in segments:
    if seg.strip():
        result = translator(seg, src_lang="en", tgt_lang="kor_Hang",num_beams=2, penalty_alpha=0.6, top_k=4,)[0]['translation_text']
    else:
        result = ""
    translations.append(result)

# save to file
with open("data/translations/AAPL_2023_Q3_translated.txt", "w", encoding="utf-8") as f:
    f.write("\n\n␟\n\n".join(translations))


In [ ]:

# 번역 실행
# korean_text = translator(en, src_lang="en", tgt_lang="kor_Hang", num_beams=2, penalty_alpha=0.6, top_k=4)

In [ ]:

# 결과 출력
# print(korean_text[0]['translation_text'])

In [ ]:
# # english_text = "Hello, world! My name is AI."
# korean_text = translator(english_text, src_lang="en", tgt_lang="kor_Hang")

In [ ]:
# en = """amd-20231230

# ␟

# UNITED STATES SECURITIES AND EXCHANGE COMMISSION

# ␟

# Washington, D.C. 20549

# ␟

# FORM 10-K

# ␟

# (Mark One)

# ␟

# For the fiscal year ended

# ␟

# OR

# ␟

# For the transition period from

# ␟

# to

# ␟

# Commission File Number

# ␟

# ADVANCED MICRO DEVICES, INC.

# ␟

# (Exact name of registrant as specified in its charter)

# ␟

# Delaware

# ␟

# (State or other jurisdiction of incorporation or organization)

# ␟

# (I.R.S. Employer  Identification No.)

# ␟

# 2485 Augustine Drive



# """

In [ ]:
# korean_text = translator(en, src_lang="en", tgt_lang="kor_Hang")
# korean_text

In [ ]:
# korean_text = translator(en, src_lang="en", tgt_lang="kor_Hang", num_beams=2, penalty_alpha=0.6, top_k=4, )
# # korean_text